# Analysis Scope

The goal of this notebook is to give an overview of how LineaPy traces code, in order to provide a mental model for understanding the scope of the code extraction, by running through a number of examples.

## Simple Expressions

Lineapy translates expressions in Python into function calls and is able to understand the dependencies between them, by tracing their arguments:

In [1]:
from lineapy import save

x = 10

y = x + 100
z = x + 200

print(save(y, "y").code)

x = 10
y = x + 100



This also works with user defined functions:

In [2]:
def my_fn(x, y):
    return x + y + 1

z = my_fn(x, y)
print(save(z, "z").code)

x = 10
y = x + 100
def my_fn(x, y):
    return x + y + 1
z = my_fn(x, y)



## Function that mutate

Many functions in Python mutate their arguments, so we cannot rely on just traversing the call graph to see what nodes are needed. 

In order to properly understand these functions, we have manually annotated which ones mutate their arguments. We have covered many builtin functions and some functions from external libraries.

For example, we know that if you call `append` on a list, this will mutate the list, so if we are saving it, we should include the append calls:

In [3]:
x = []
x.append(10)
print(save(x, "x").code)

x = []
x.append(10)



The mutation tracking also differentiates between calls before and after a mutation, for example calling `len` before appending should not include the append call, but after should!

In [4]:
x = []
early_len = len(x)
x.append(10)
later_len = len(x)

print("Early:")
print(save(early_len, "early_len").code)
print("Later:")
print(save(later_len, "later_len").code)

Early:
x = []
early_len = len(x)

Later:
x = []
x.append(10)
later_len = len(x)



This means that if you are calling some function which mutates one of its args, and we haven't annotated that behavior, lineapy won't know about it and it might be lost when slicing!

## Tracking "views"

When an object is "mutated" we also consider all other objects which have references to it as having "mutated" as well. This is because their behavior might change after mutating this object.

Consider a dictionary with a list in it as a key:

In [5]:
l = []
d = {"a": l}

And we want to get the sum of the length of all values in the dict:

In [6]:
early_len = sum(map(len, d.values()))
early_len

0

In [7]:
print(save(early_len, "early_len").code)

l = []
d = {"a": l}
early_len = sum(map(len, d.values()))



Now we add an item to that list.

If we get the length after this, the value will be different:

In [8]:
l.append(10)
later_len = sum(map(len, d.values()))
later_len

1

So if we slice on the `later_len` it needs to include the code for the append to be accurate:

In [9]:
print(save(later_len, "later_len").code)

l = []
d = {"a": l}
l.append(10)
later_len = sum(map(len, d.values()))



We are able to track this by also annotating function if they add any "views" between their arguments or results. We consider views to be bidirectional, and if two objects are views of one another, a mutation of one will mutate the other.